# Preprocessing and registration
This notebook does basic preprocessing and registration dragonfly multiresolution data

# Notes
I think I could improve results by padding the low res image with black on either side. I did this with the atlas mapping too

Note, I think that for the high res channel 0 is cell label. for high res, channel 1 is cell label

# setting up for tme08

v00 is just getting set up

# Andrew's Notes

Only variables that need to be modified:
- brain = {TME08-1, TME10-1, ...}
- slice_ = {'01A', '01B', '02A', ...}
- A0: Initial guess of affine matrix
    - Line up 10x (pink) and 30x (green) as closely as possible, then run optimization procedure

Slices in each brain:
- TME08-1: 01A, ..., 08B
- (Completed by Daniel) TME09-1: 01A, ..., 08B
- TME10-1: 01A, ..., 08B
- TME10-3: 01A, ..., 09A
- TME12-1: 01A, ..., 08B
- TME20-1: 01A, ..., 08B

## Import Necessary Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib notebook
%matplotlib widget

import time
import torch
from glob import glob

import os
from os.path import join,split,splitext

import sys
sys.path.append('..')
# sys.path.append('/ifshome/abenneck')
sys.path.append('/home/abenneck')
import importlib as imp
import donglab_workflows as dw
imp.reload(dw)

# # for now add emlddmm library for registration
from emlddmm import emlddmm
imp.reload(emlddmm)

from scipy.interpolate import interpn

## Define Input and Output Paths

Lists of all potential inputs for 'brain':
- [TME08-1, TME10-1, TME10-3, TME12-1, TME20-1]
- [Hpca5-2, Sp9-3-2, Trank1-2-3, Zswim4-1]
- [MQC06-2, MQC09-3, MQC18-3, MQC82-2]

In [ ]:
# where should outputs go?
# brain = 'TME07-1'
# brain = 'TME08-1'
# brain = 'TME09-1'
# brain = 'TME10-1'
# brain = 'TME10-3'
# brain = 'TME12-1'
# brain = 'TME20-1'
# brain = 'Hpca5-2'
# brain = 'Sp9-3-2'
# brain = 'Trank1-2-3'
# brain = 'Zswim4-1'
# brain = 'MQC06-2'
# brain = 'MQC09-3'
# brain = 'MQC18-3'
# brain = 'MQC82-2'
# brain = 'hTME15-1'
# brain = 'hTME15-2'
# brain = 'hTME18-1'
# brain = 'hTME19-2'
brain = '12m_MQC06-2'
# brain = '12m_MQC15-1'
# brain = '12m_MQC18-3'
# brain = '12m_MQC07-5'
# brain = '12m_MQC09-3'
# brain = '12m_MQC6-4'

if brain in ['TME08-1', 'TME10-1', 'TME10-3', 'TME12-1', 'TME20-1', 'TME07-1', 'TME09-1']:
    brain_path = brain
elif brain in ['MQC06-2', 'MQC09-3', 'MQC18-3', 'MQC82-2']: # The original HD brains
    brain_path = f'Q140_MORF/{brain}'
elif '12m' in brain: # The new HD brains
    if '06-2' in brain or '15-1' in brain or '18-3' in brain:
        brain_path = f'Q140_MORF/Shasha_Q140/Camk-MORF3_{brain}'
    elif '07-5' in brain or '09-3' in brain:
        brain_path = f'Q140_MORF/Shasha_Q140/Camk-MORF3-Q140_{brain}'
    elif '6-4' in brain:
        brain_path = f'Q140_MORF/Shasha_Q140/Camk-MORF3-Q140_12m_MQC6-4 (prev 6-3)'
    else:
        raise Exception('Invalid brain')
elif 'hTME' in brain:
    if 'hTME15-1' in brain or 'hTME19-2' in brain:
        brain_path = f'Q140_MORF_D1/Camk-MORF3-D1Tom_12m_{brain}'
    elif 'hTME15-2' in brain or 'hTME18-1' in brain:
        brain_path = f'Q140_MORF_D1/Camk-MORF3-D1Tom-Q140_12m_{brain}'
    else:
        raise Exception('Invalid brain')
else: # KO-Het_MORF
    if 'Hpca' in brain:
        brain_path = f'KO-Het_MORF/HpcaKO-Camk-MORF3-D1tom_Hpca5-2'
    elif 'Sp9' in brain:
        brain_path = f'KO-Het_MORF/Sp9Het-Camk-MORF3-D1tom_Sp9-3-2'
    elif 'Trank' in brain:
        brain_path = f'KO-Het_MORF/Trank1KO-Camk-MORF3-D1tom_Trank1-2-3'
    elif 'Zswim' in brain:
        brain_path = f'KO-Het_MORF/Zswim6Het-Camk-MORF3-D1tom_Zswim4-1'
    else:
        raise Exception('Invalid brain')

output_prefix = f'/home/abenneck/dragonfly_work/dragonfly_outputs/{brain}/dragonfly_10_to_30_outputs/'

# output_directory = split(output_prefix)[0]
# if output_directory:
#     os.makedirs(output_directory,exist_ok=True)

### Slices / brain
- hTME15-1: [01, ..., 08]
- hTME15-2: [01, ..., 08]
- hTME18-1: [01, ..., 08]
- hTME19-2: [01, ..., 08] (01a and 01b?)
- 12m_MQC06-2: [1,2,3,4,5,6]
- 12m_MQC15-1: [3,4,5,6,7]
- 12m_MQC18-3: [1,2,3,4,5,6,7]
- 12m_MQC07-5: [1,2,3,4,5,6]
- 12m_MQC09-3: [1,2,3,4,5,6,7]
- 12m_MQC6-4: [2,3,4,5,6]

In [ ]:
slice_ = '01A'
# slice_ = '01B'
# slice_ = '02A'
# slice_ = '02B'
# slice_ = '03A'
# slice_ = '03B'
# slice_ = '04A'
# slice_ = '04B'
# slice_ = '05A'
# slice_ = '05B'
# slice_ = '06A'
# slice_ = '06B'
# slice_ = '07A'
# slice_ = '07B'
# slice_ = '08A'
# slice_ = '08B'
# slice_ = '09A'
# slice_ = '09B'

if 'TME07-1' in brain and '08B' in slice_:
    channel = 1
else:
    channel = 0 # May be channel 1 for low-res

target_dir = f'/panfs/dong/3D_registration/Yang_MORF_DragonFly/{brain_path}_downsampled/30x/ch_{channel}_pipeline_building/'

if 'TME' in brain and '07' not in brain and 'hTME' not in brain:
    target_name = join(target_dir,f'Camk2a-MORF3-D1Tom_{brain}_30x_Str_{slice_}_ch_{channel}_pow_[0.125]_down.npz')
elif 'TME07-1' in brain:
    target_dir = '/nafs/dtward/dong/dragonfly_03_2022'
    target_pattern = f'Camk2a-MORF3-D1Tom_TME07-1_30x_Str_{slice_}_channel_{channel}_down.npz'
    target_name = glob(join(target_dir,target_pattern))[0]
elif '12m' in brain:
    target_pattern_0 = f'*S{slice_[1:-1]}*30X*.npz'
    target_pattern_1 = f'*S{slice_[1:-1]}*30x*.npz'
    target_pattern_2 = f'*s{slice_[1:-1]}*30x*.npz'
    target_pattern_3 = f'*s{slice_[1:-1]}*30X*.npz'
    target_pattern_4 = f'*30X*S{slice_[1:-1]}*.npz'
    if len(glob(join(target_dir,target_pattern_0))) != 0:
        target_name = glob(join(target_dir,target_pattern_0))[0]
    elif len(glob(join(target_dir,target_pattern_1))) != 0:
        target_name = glob(join(target_dir,target_pattern_1))[0]    
    elif len(glob(join(target_dir,target_pattern_2))) != 0:
        target_name = glob(join(target_dir,target_pattern_2))[0]     
    elif len(glob(join(target_dir,target_pattern_3))) != 0:
        target_name = glob(join(target_dir,target_pattern_3))[0] 
    elif len(glob(join(target_dir,target_pattern_4))) != 0:
        target_name = glob(join(target_dir,target_pattern_4))[0] 
    else:
        raise Exception('Invalid slice')
else:
    target_pattern = f'*30X*Str_{slice_[:-1]}*.npz'
    if len(glob(join(target_dir,target_pattern))) == 0:
        target_pattern = f'*30x*Str_{slice_[:-1]}*.npz'
        target_name = glob(join(target_dir,target_pattern))[0]
    else:
        target_name = glob(join(target_dir,target_pattern))[0]
            
# elif ('12m' not in brain) and ('MQC06' in brain or 'MQC09' in brain or 'MQC82' in brain or 'hTME' in brain):
#     target_pattern = f'*30X*Str_{slice_[:-1]}*.npz'
#     target_name = glob(join(target_dir,target_pattern))[0]
# # elif:
    
# else:
#     target_pattern = f'*30x*Str_{slice_[:-1]}*.npz'
#     target_name = glob(join(target_dir,target_pattern))[0]

# This slice has 2 files and is an exception
if brain == 'hTME19-2' and slice_ == '01B':
    target_name = '/panfs/dong/3D_registration/Yang_MORF_DragonFly/Q140_MORF_D1/Camk-MORF3-D1Tom_12m_hTME19-2_downsampled/30x/ch_0_pipeline_building/hTME19-2_Camk-MORF3-D1Tom_12m_30X_Str_01b_2023-05-18_18.22.18_FusionbStitcher_ch_0_pow_[0.125]_down.npz'

target_name

In [ ]:
target_data = np.load(target_name,allow_pickle=True)

[k for k in target_data]

In [ ]:
J = target_data['I'][None]

if 'TME07-1' in brain:
    W = target_data['W']
else:
    W = target_data['w']

J = J.astype(np.float32)

if channel == 1:
    J -= np.quantile(J[W[None]>0.9],0.01)
    J /= np.quantile(J[W[None]>0.9],0.999)
    J[W[None]<=0.90] = 0.0 # there should be nothing less than 0
elif channel == 0:
    J -= np.quantile(J[W[None]>0.9],0.1)
    J /= np.quantile(J[W[None]>0.9],0.999)
    J[W[None]<=0.90] = 0.0 # there should be nothing less than 0
    
J[J<0] = 0.0
if 'TME' in brain:
    xJ = [d - np.mean(d) for d in target_data['xI']]
else:
    xJ = target_data['xI']
    
dJ = [x[1] - x[0] for x in xJ]
J0 = np.copy(J)

# Normalization, so that most of the data is [0,1]

In [ ]:
fig,ax = plt.subplots()
ax.hist(J.ravel(),bins=100)

In [ ]:
fig,ax = emlddmm.draw(J,xJ,vmin=0,vmax=1)
fig.suptitle('Downsampled lightsheet data')
# figopts = {'dpi':300,'quality':90}
figopts = {'dpi':300}
fig.savefig(output_prefix + '_down.jpg',**figopts)
fig.canvas.draw()

# Grid is an artifact emerging due to overstimaultion of tissue

In [ ]:
if 'TME' in brain:
    # target_key = splitext(split(target_name)[-1])[0].split('_')[-6]
    target_key = slice_
else:
    target_key = slice_[:-1]
target_key

In [ ]:
if 'TME07-1' in brain and '08B' in slice_:
    atlas_channel = 0
else:
    atlas_channel = 1

if 'TME07-1' in brain:
    atlas_names = glob(f'/nafs/dtward/dong/dragonfly_03_2022/*10x_{slice_[:-1]}*_channel_{atlas_channel}*.npz')
elif 'TME' in brain and 'hTME' not in brain:
    atlas_names = glob(f'/panfs/dong/3D_registration/Yang_MORF_DragonFly/{brain_path}_downsampled/10x/ch_{atlas_channel}_pipeline_building/*{slice_[:-1]}*ch_{atlas_channel}*.npz')
elif '12m' not in brain and ('MQC06' in brain or 'MQC09' in brain or 'MQC82' in brain or 'hTME' in brain):
    atlas_names = glob(f'/panfs/dong/3D_registration/Yang_MORF_DragonFly/{brain_path}_downsampled/10x/ch_{atlas_channel}_pipeline_building/*10X_{slice_[:-1]}*_ch_{atlas_channel}*.npz')    
elif '12m' in brain:
    target_dir = f'/panfs/dong/3D_registration/Yang_MORF_DragonFly/{brain_path}_downsampled/10x/ch_{atlas_channel}_pipeline_building/'
    target_pattern_0 = f'*S{slice_[1:-1]}*10X*.npz'
    target_pattern_1 = f'*S{slice_[1:-1]}*10x*.npz'
    target_pattern_2 = f'*s{slice_[1:-1]}*10x*.npz'
    target_pattern_3 = f'*s{slice_[1:-1]}*10X*.npz'
    target_pattern_4 = f'*10X*S{slice_[1:-1]}*.npz'
    if len(glob(join(target_dir,target_pattern_0))) != 0:
        atlas_names = glob(join(target_dir,target_pattern_0))
    elif len(glob(join(target_dir,target_pattern_1))) != 0:
        atlas_names = glob(join(target_dir,target_pattern_1))
    elif len(glob(join(target_dir,target_pattern_2))) != 0:
        atlas_names = glob(join(target_dir,target_pattern_2))
    elif len(glob(join(target_dir,target_pattern_3))) != 0:
        atlas_names = glob(join(target_dir,target_pattern_3))
    elif len(glob(join(target_dir,target_pattern_4))) != 0:
        atlas_names = glob(join(target_dir,target_pattern_4))
    else:
        raise Exception('Invalid slice')
else:
    atlas_names = glob(f'/panfs/dong/3D_registration/Yang_MORF_DragonFly/{brain_path}_downsampled/10x/ch_{atlas_channel}_pipeline_building/*10x_{slice_[:-1]}*_ch_{atlas_channel}*.npz')
atlas_names # Should only have 1 element

In [ ]:
if brain == 'hTME19-2' and slice_ == '01A':
    atlas_names = [atlas_names[0]]
elif brain == 'hTME19-2' and slice_ == '01B':
    atlas_names = [atlas_names[1]]

I = []
for atlas_name in atlas_names:
    atlas_data = np.load(atlas_name,allow_pickle=True)
    if 'TME' in brain:
        xI = [d - np.mean(d) for d in atlas_data['xI']]
    else:
        xI = atlas_data['xI']

    I_ = atlas_data['I'].astype(np.float32)    
    
    if 'TME07-1' in brain:
        I_ -= np.quantile(I_[atlas_data['W']>0.9],0.1)
        I_ /= np.quantile(I_[atlas_data['W']>0.9],0.999)      
        I_[atlas_data['W']<=0.9] = 0.0
    else:
        I_ -= np.quantile(I_[atlas_data['w']>0.9],0.1)
        I_ /= np.quantile(I_[atlas_data['w']>0.9],0.999)
        I_[atlas_data['w']<=0.9] = 0.0
        
    I.append(I_)
    
    
I = np.concatenate(I)    
if I.ndim == 3:
    I = I[None]
dI = [x[1] - x[0] for x in xI]
XI = np.meshgrid(*xI,indexing='ij')

In [ ]:
fig,ax = emlddmm.draw(I,xI,vmin=0.0,vmax=1)
fig.canvas.draw()

# Initial preprocessing



## (GAMMA CORRECTION) Target (30x) preprocessing

In [ ]:
# Special cases which had nonuniform brightness across slices
def gammaCorrectionNecessary(brain, sliceNum):
    if 'Trank' in brain and ('04' in sliceNum or '05' in sliceNum):
        return True
    elif 'MQC82-2' in brain:
        return True
    elif 'MQC09-3' in brain and ('03' in sliceNum or '07' in sliceNum):
        return True
    elif 'TME07-1' in brain and '08B' in sliceNum:
        return True
    elif 'hTME15-1' in brain and '08' in sliceNum:
        return True
    elif 'hTME15-2' in brain and '02' in sliceNum:
        return True
    elif 'hTME18-1' in brain and '08' in sliceNum:
        return True
    elif 'hTME19-2' in brain and '01B' in sliceNum:
        return True
    elif '12m_MQC06-2' in brain and '2' in sliceNum:
        return True
    else:
        return False

In [ ]:
# for target, only pad in the slice direction
npad = 8
J = np.pad(J,  ((0,0),(npad,npad),(0,0),(0,0)) )
#W = np.pad(W,  ((npad,npad),(0,0),(0,0)) , mode='edge') 
W = np.pad(W,  ((npad,npad),(0,0),(0,0)) , mode='constant',constant_values=1.0) 
for i in range(npad):    
    xJ = [ np.concatenate( (xJ[0][0][None]-dJ[0], xJ[0],xJ[0][-1][None]+dJ[0])), xJ[1], xJ[2]  ]

if gammaCorrectionNecessary(brain, slice_):
    # Gamma correction for 30x images with nonuniform brightness
    # for step 2 we will want to do some contrast adjustment on the 30x high res images,
    # especially (only?) for the slices that fail
    n = J.shape[1]
    # let's start with something simple
    # we'll start with a gamma correction to reduce the contrast in the brightest pixels
    nrow = np.ceil(np.sqrt(n)).astype(int)
    Jmin = np.min(J[W[None]>0.9])
    Jmax = np.max(J[W[None]>0.9])
    J_ = np.copy(J)
    J_ -= Jmin
    J_ /= Jmax-Jmin
    J_ = np.clip(J_,0,1)
    J_ = J_**0.5
    # fig,ax = plt.subplots(nrow,nrow)
    # ax = ax.ravel()
    # for i in range(n):
    #     h = ax[i].imshow(J_[i],vmin=0,vmax=1)
    # plt.colorbar(mappable=h,ax=ax)

    J = J_

In [ ]:
# adjust dynamic range
#J[J<0]=0
#J = J**0.5
# adjust mean value
#J /= np.mean(np.abs(J))
fig,ax = emlddmm.draw(J,xJ,vmin=0,vmax=1)
fig.canvas.draw()
fig,ax = plt.subplots()
ax.hist(J.ravel(),100,log=True)
fig.canvas.draw()

## Atlas (10x) preprocessing


In [ ]:
# pad
# since I will downsample by 4, I want to pad with 4x4x4
npad = 4
I = np.pad(I,  ((0,0),(npad,npad),(npad,npad),(npad,npad)) )
for i in range(npad):
    xI = [ np.concatenate(   (x[0][None]-d,x,x[-1][None]+d)   ) for d,x in zip(dI,xI)]


In [ ]:
# adjust dynamic range
I[I<0]=0
#I = I**0.5
# adjust mean value
#J /= np.mean(np.abs(J))
fig,ax = emlddmm.draw(I,xI,vmin=0,vmax=1)
fig.canvas.draw()
fig,ax = plt.subplots()
ax.hist(I.ravel(),100,log=True)
fig.canvas.draw()

# Initial Guess

## Setup

In [ ]:
# initial affine
# make sure to keep sign of Jacobian

A0 = np.eye(4)

# the slice direction doesn't seem quite right, but the others do
A0[0,-1] = np.mean(xJ[0]) - np.mean(xI[0])
# NOTE, this didn't work with our current downsampling

# 1000 and 2000 are shifts in microns (Example starting shifts for slices *A and *B)
if False:
    if 'A' in slice_:
        A0[:3,-1] = np.array([0,1000,2000])
    if 'B' in slice_:
        A0[:3,-1] = np.array([0,1000,-2000])

### Initial guesses for other brains

In [ ]:
# For all brains ([a,b,c] shifts), change b to translate vertically and change c to translate horizontally

if 'TME07-1' in brain:
    if '01A' in slice_: # Skipped due to error in image
        A0[:3,-1] = np.array([0,500,-1500])    
    if '01B' in slice_:
        A0[:3,-1] = np.array([0,400,-1600])   
    if '02A' in slice_:
        A0[:3,-1] = np.array([0,800,1600])  
    if '02B' in slice_:
        A0[:3,-1] = np.array([0,900,-1400])        
    if '03A' in slice_:
        A0[:3,-1] = np.array([0,700,1800])
    if '03B' in slice_:
        A0[:3,-1] = np.array([0,600,-1400])  
    if '04A' in slice_:
        A0[:3,-1] = np.array([0,500,2000])          
    if '04B' in slice_:
        A0[:3,-1] = np.array([0,500,-1700])
    if '05A' in slice_:
        A0[:3,-1] = np.array([0,500,2200])
    if '05B' in slice_:
        A0[:3,-1] = np.array([0,600,-2400])
    if '06A' in slice_:
        A0[:3,-1] = np.array([0,0,3200])
    if '06B' in slice_:
        A0[:3,-1] = np.array([0,500,-2300])        
    if '07A' in slice_:
        A0[:3,-1] = np.array([0,200,3500])
    if '07B' in slice_:
        A0[:3,-1] = np.array([0,600,-3000]) 
    if '08A' in slice_:
        A0[:3,-1] = np.array([0,200,3800])        
    if '08B' in slice_: # Inaccurate due to damaged 30x tissue
        A0[:3,-1] = np.array([0,200,-3300]) 

if 'TME08-1' in brain:
    if '01A' in slice_:
        A0[:3,-1] = np.array([0,1000,2000])
    if '01B' in slice_:
        A0[:3,-1] = np.array([0,1000,-1100])
    if '02A' in slice_:
        A0[:3,-1] = np.array([0,500,2000])
    if '02B' in slice_:
        A0[:3,-1] = np.array([0,600,-1800])
    if '03A' in slice_:
        A0[:3,-1] = np.array([0,500,2700])
    if '03B' in slice_:
        A0[:3,-1] = np.array([0,500,-1900])
    if '04A' in slice_:
        A0[:3,-1] = np.array([0,800,2900])    
    if '04B' in slice_:
        A0[:3,-1] = np.array([0,1000,-2000])
    if '05A' in slice_:
        A0[:3,-1] = np.array([0,300,3200])
    if '05B' in slice_:
        A0[:3,-1] = np.array([0,100,-2700])
    if '06A' in slice_:
        A0[:3,-1] = np.array([0,500,3300])
    if '06B' in slice_:
        A0[:3,-1] = np.array([0,300,-3200])    
    if '07A' in slice_:
        A0[:3,-1] = np.array([0,300,3500])        
    if '07B' in slice_:
        A0[:3,-1] = np.array([0,200,-3300])
    if '08A' in slice_:
        A0[:3,-1] = np.array([0,500,3200])
    if '08B' in slice_:
        A0[:3,-1] = np.array([0,300,-3800])
        
if 'TME09-1' in brain:
    if '01A' in slice_:
        A0[:3,-1] = np.array([0,1100,1500])
    if '01B' in slice_:
        A0[:3,-1] = np.array([0,700,-1200])
    if '02A' in slice_:
        A0[:3,-1] = np.array([0,600,2200])
    if '02B' in slice_:
        A0[:3,-1] = np.array([0,700,-1200])        
    if '03A' in slice_:
        A0[:3,-1] = np.array([0,600,2200])        
    if '03B' in slice_:
        A0[:3,-1] = np.array([0,500,-2000])     
    if '04A' in slice_:
        A0[:3,-1] = np.array([0,600,2200])             
    if '04B' in slice_:
        A0[:3,-1] = np.array([0,500,-2400])  
    if '05A' in slice_:
        A0[:3,-1] = np.array([0,300,2800])      
    if '05B' in slice_:
        A0[:3,-1] = np.array([0,300,-2200])  
    if '06A' in slice_:
        A0[:3,-1] = np.array([0,800,3100])              
    if '06B' in slice_:
        A0[:3,-1] = np.array([0,700,-2700])
    if '07A' in slice_:
        A0[:3,-1] = np.array([0,900,3300])           
    if '07B' in slice_:
        A0[:3,-1] = np.array([0,900,-3000])
    if '08A' in slice_:
        A0[:3,-1] = np.array([0,1000,3600])          
    if '08B' in slice_:
        A0[:3,-1] = np.array([0,1100,-3200])
        
if 'TME10-1' in brain:
    if '01A' in slice_:
        A0[:3,-1] = np.array([0,700,1500])    
    if '01B' in slice_:
        A0[:3,-1] = np.array([0,600,-1200])
    if '02A' in slice_:
        A0[:3,-1] = np.array([0,900,2100])        
    if '02B' in slice_:
        A0[:3,-1] = np.array([0,1000,-1500])
    if '03A' in slice_:
        A0[:3,-1] = np.array([0,500,2200])     
    if '03B' in slice_:
        A0[:3,-1] = np.array([0,500,-1800])        
    if '04A' in slice_:
        A0[:3,-1] = np.array([0,400,2400])             
    if '04B' in slice_:
        A0[:3,-1] = np.array([0,400,-2300])   
    if '05A' in slice_:
        A0[:3,-1] = np.array([0,500,2900])        
    if '05B' in slice_:
        A0[:3,-1] = np.array([0,400,-2100])
    if '06A' in slice_:
        A0[:3,-1] = np.array([0,800,3100])
    if '06B' in slice_:
        A0[:3,-1] = np.array([0,700,-2700])
    if '07A' in slice_:
        A0[:3,-1] = np.array([0,1000,3700])
    if '07B' in slice_:
        A0[:3,-1] = np.array([0,800,-3000])
    if '08A' in slice_:
        A0[:3,-1] = np.array([0,600,3600])
    if '08B' in slice_:
        A0[:3,-1] = np.array([0,800,-3200])
        
if 'TME10-3' in brain:
    if '01A' in slice_:
        A0[:3,-1] = np.array([0,600,1300])
    if '01B' in slice_:
        A0[:3,-1] = np.array([0,400,-1200])        
    if '02A' in slice_:
        A0[:3,-1] = np.array([0,700,1600])        
    if '02B' in slice_:
        A0[:3,-1] = np.array([0,700,-1500])        
    if '03A' in slice_:
        A0[:3,-1] = np.array([0,500,2100])
    if '03B' in slice_:
        A0[:3,-1] = np.array([0,500,-1600])        
    if '04A' in slice_:
        A0[:3,-1] = np.array([0,800,2300])        
    if '04B' in slice_:
        A0[:3,-1] = np.array([0,800,-1700])
    if '05A' in slice_:
        A0[:3,-1] = np.array([0,500,2500])        
    if '05B' in slice_:
        A0[:3,-1] = np.array([0,400,-2200])        
    if '06A' in slice_:
        A0[:3,-1] = np.array([0,600,2700])
    if '06B' in slice_:
        A0[:3,-1] = np.array([0,500,-2700])        
    if '07A' in slice_:
        A0[:3,-1] = np.array([0,800,2800])
    if '07B' in slice_:
        A0[:3,-1] = np.array([0,800,-3000])
    if '08A' in slice_:
        A0[:3,-1] = np.array([0,1000,3700])        
    if '08B' in slice_:
        A0[:3,-1] = np.array([0,1200,-2700])        
    if '09A' in slice_:
        A0[:3,-1] = np.array([0,1000,3900])   
        
if 'TME12-1' in brain:
    if '01A' in slice_:
        A0[:3,-1] = np.array([0,0,1800])
    if '01B' in slice_:
        A0[:3,-1] = np.array([0,-200,-1000]) 
    if '02A' in slice_:
        A0[:3,-1] = np.array([0,500,1400])
    if '02B' in slice_:
        A0[:3,-1] = np.array([0,700,-1300])         
    if '03A' in slice_:
        A0[:3,-1] = np.array([0,100,2200])
    if '03B' in slice_:
        A0[:3,-1] = np.array([0,0,-1200])
    if '04A' in slice_:
        A0[:3,-1] = np.array([0,-100,2500])        
    if '04B' in slice_:
        A0[:3,-1] = np.array([0,0,-1700])        
    if '05A' in slice_:
        A0[:3,-1] = np.array([0,100,2700])    
    if '05B' in slice_:
        A0[:3,-1] = np.array([0,-100,-2100])   
    if '06A' in slice_:
        A0[:3,-1] = np.array([0,0,2800])   
    if '06B' in slice_:
        A0[:3,-1] = np.array([0,-100,-2700])           
    if '07A' in slice_:
        A0[:3,-1] = np.array([0,-100,2900]) 
    if '07B' in slice_:
        A0[:3,-1] = np.array([0,400,-3000]) 
    if '08A' in slice_:
        A0[:3,-1] = np.array([0,200,3100]) 
    if '08B' in slice_:
        A0[:3,-1] = np.array([0,100,-3200])
        
if 'TME20-1' in brain:
    if '01A' in slice_:
        A0[:3,-1] = np.array([0,500,1400])
    if '01B' in slice_:
        A0[:3,-1] = np.array([0,700,-1200])
    if '02A' in slice_:
        A0[:3,-1] = np.array([0,800,2200])        
    if '02B' in slice_:
        A0[:3,-1] = np.array([0,700,-1300])
    if '03A' in slice_:
        A0[:3,-1] = np.array([0,700,2100])            
    if '03B' in slice_:
        A0[:3,-1] = np.array([0,500,-2200])
    if '04A' in slice_:
        A0[:3,-1] = np.array([0,200,2600])  
    if '04B' in slice_:
        A0[:3,-1] = np.array([0,300,-2100])
    if '05A' in slice_:
        A0[:3,-1] = np.array([0,0,2600])  
    if '05B' in slice_:
        A0[:3,-1] = np.array([0,0,-2200])        
    if '06A' in slice_:
        A0[:3,-1] = np.array([0,0,3200])  
    if '06B' in slice_:
        A0[:3,-1] = np.array([0,100,-2500])   
    if '07A' in slice_:
        A0[:3,-1] = np.array([0,300,3400])          
    if '07B' in slice_:
        A0[:3,-1] = np.array([0,200,-3200]) 
    if '08A' in slice_:
        A0[:3,-1] = np.array([0,200,3600])
    if '08B' in slice_:
        A0[:3,-1] = np.array([0,200,-3500])  
        
if 'Hpca5-2' in brain:
    if '01' in slice_:
        A0[:3,-1] = np.array([0,-500,-500])
    if '02' in slice_:
        A0[:3,-1] = np.array([0,-300,-1100])
    if '03' in slice_:
        A0[:3,-1] = np.array([0,200,-700])
    if '04' in slice_:
        A0[:3,-1] = np.array([0,-1300,-1000])
    if '05' in slice_:
        A0[:3,-1] = np.array([0,0,-1400])
    if '06' in slice_:
        A0[:3,-1] = np.array([0,-1100,-1200])
    if '07' in slice_:
        A0[:3,-1] = np.array([0,-700,-1100])
        
if 'Sp9-3-2' in brain:
    if '01' in slice_:
        A0[:3,-1] = np.array([0,-2000,-1000])  
    if '02' in slice_:
        A0[:3,-1] = np.array([0,-1500,-1500])
    if '03' in slice_:
        A0[:3,-1] = np.array([0,-1200,-1100])
    if '04' in slice_:
        A0[:3,-1] = np.array([0,-1200,-1100])
    if '05' in slice_:
        A0[:3,-1] = np.array([0,-1200,-1100])
    if '06' in slice_:
        A0[:3,-1] = np.array([0,-1200,-1100])
    if '07' in slice_:
        A0[:3,-1] = np.array([0,-1900,-1100])
    if '08' in slice_:
        A0[:3,-1] = np.array([0,-1900,-1100])
        
if 'Trank1-2-3' in brain:
    if '01' in slice_:
        A0[:3,-1] = np.array([0,-200,-1500])
    if '02' in slice_:
        A0[:3,-1] = np.array([0,-1200,-1400])    
    if '03' in slice_:
        A0[:3,-1] = np.array([0,-1300,-900])  
    if '04' in slice_:
        A0[:3,-1] = np.array([0,-900,-1000])  
    if '05' in slice_:
        A0[:3,-1] = np.array([0,-900,-1000])          
    if '06' in slice_:
        A0[:3,-1] = np.array([0,-900,-1000])           
    if '07' in slice_:
        A0[:3,-1] = np.array([0,-300,-900])     
    if '08' in slice_:
        A0[:3,-1] = np.array([0,-200,-1000])
        
if 'Zswim4-1' in brain:
    if '01' in slice_:
        A0[:3,-1] = np.array([0,-200,-1200])   
    if '02' in slice_:
        A0[:3,-1] = np.array([0,200,-1200])           
    if '03' in slice_:
        A0[:3,-1] = np.array([0,-1000,-1100])   
    if '04' in slice_:
        A0[:3,-1] = np.array([0,-800,-1000])           
    if '05' in slice_:
        A0[:3,-1] = np.array([0,-800,-1000])   
    if '06' in slice_:
        A0[:3,-1] = np.array([0,-1000,-1100])        
    if '07' in slice_:
        A0[:3,-1] = np.array([0,-400,-1100])  
    if '08' in slice_:
        A0[:3,-1] = np.array([0,-700,-900])
        
if 'MQC06-2' == brain:
    if '01' in slice_:
        A0[:3,-1] = np.array([0,-200,-1200])
    if '02' in slice_:
        A0[:3,-1] = np.array([0,200,-1000])
    if '03' in slice_:
        A0[:3,-1] = np.array([0,-100,-800])        
    if '04' in slice_:
        A0[:3,-1] = np.array([0,100,-800])  
    if '05' in slice_:
        A0[:3,-1] = np.array([0,-1200,-800])  
    if '06' in slice_:
        A0[:3,-1] = np.array([0,-900,-1000])  
    if '07' in slice_:
        A0[:3,-1] = np.array([0,-300,-1000])  
    if '08' in slice_:
        A0[:3,-1] = np.array([0,-500,-1000]) 
        
if 'MQC09-3' == brain:
    if '01' in slice_:
        A0[:3,-1] = np.array([0,-1100,-1000])     
    if '02' in slice_:
        A0[:3,-1] = np.array([0,-500,-800])   
    if '03' in slice_: # Poor 30x image quality
        A0[:3,-1] = np.array([0,-300, -800])          
    if '04' in slice_:
        A0[:3,-1] = np.array([0,-300,-800])  
    if '05' in slice_:
        A0[:3,-1] = np.array([0,-200,-800])  
    if '06' in slice_:
        A0[:3,-1] = np.array([0,0,-800])  
    if '07' in slice_: # Poor 30x image quality
        A0[:3,-1] = np.array([0,-100,-800])          
    if '08' in slice_:
        A0[:3,-1] = np.array([0,0,-700])  
        
if 'MQC18-3' == brain:
    if '01' in slice_:
        A0[:3,-1] = np.array([0,-800,-800])
    if '02' in slice_:
        A0[:3,-1] = np.array([0,-600,-800])
    if '03' in slice_:
        A0[:3,-1] = np.array([0,-600,-800])
    if '04' in slice_:
        A0[:3,-1] = np.array([0,-200,-1000])
    if '05' in slice_:
        A0[:3,-1] = np.array([0,-400,-800])
    if '06' in slice_:
        A0[:3,-1] = np.array([0,-400,-1000])        
    if '07' in slice_:
        A0[:3,-1] = np.array([0,-1400,-1000])  
    if '08' in slice_:
        A0[:3,-1] = np.array([0,-200,-1000])  
        
if 'MQC82-2' == brain:
    if '01' in slice_:
        A0[:3,-1] = np.array([0,-600,-1000])
    if '02' in slice_:
        A0[:3,-1] = np.array([0,-500,-1000])
    if '03' in slice_: 
        A0[:3,-1] = np.array([0,-100,-900])
    if '04' in slice_:
        A0[:3,-1] = np.array([0,-100, -900])
    if '05' in slice_:
        A0[:3,-1] = np.array([0,600, -1500])
    if '06' in slice_:
        A0[:3,-1] = np.array([0,-900, -600])
    if '07' in slice_:
        A0[:3,-1] = np.array([0,-500,-1000])
    if '08' in slice_:
        A0[:3,-1] = np.array([0,-500,-1000])

if 'hTME15-1' in brain:
    if '01' in slice_:
        A0[:3,-1] = np.array([0,-300,200])    
    if '02' in slice_:
        A0[:3,-1] = np.array([0,-500,200])  
    if '03' in slice_:
        A0[:3,-1] = np.array([0,-500,200])  
    if '04' in slice_:
        A0[:3,-1] = np.array([0,-200,200]) 
    if '05' in slice_:
        A0[:3,-1] = np.array([0,-200,300]) 
    if '06' in slice_:
        A0[:3,-1] = np.array([0,300,100]) 
    if '07' in slice_:
        A0[:3,-1] = np.array([0,200,-400]) 
    if '08' in slice_: # NOT ALIGNED DUE TO IMAGE QUALITY
        A0[:3,-1] = np.array([0,-600,-200]) 

if 'hTME15-2' in brain:
    if '01' in slice_:
        A0[:3,-1] = np.array([0,300,500])    
    if '02' in slice_:
        A0[:3,-1] = np.array([0,-500,200])    
    if '03' in slice_:
        A0[:3,-1] = np.array([0,-500,100]) 
    if '04' in slice_:
        A0[:3,-1] = np.array([0,-400,-300]) 
    if '05' in slice_:
        A0[:3,-1] = np.array([0,0,-300]) 
    if '06' in slice_:
        A0[:3,-1] = np.array([0,300,600]) 
    if '07' in slice_:
        A0[:3,-1] = np.array([0,-100,-600]) 
    if '08' in slice_:
        A0[:3,-1] = np.array([0,-400,-600]) 

if 'hTME18-1' in brain:
    if '01' in slice_:
        A0[:3,-1] = np.array([0,0,300])    
    if '02' in slice_:
        A0[:3,-1] = np.array([0,100,0])  
    if '03' in slice_:
        A0[:3,-1] = np.array([0,-200,200])  
    if '04' in slice_:
        A0[:3,-1] = np.array([0,-200,0])  
    if '05' in slice_:
        A0[:3,-1] = np.array([0,200,-200]) 
    if '06' in slice_:
        A0[:3,-1] = np.array([0,100,-400]) 
    if '07' in slice_:
        A0[:3,-1] = np.array([0,300,-600]) 
    if '08' in slice_:
        A0[:3,-1] = np.array([0,-300,-600]) 

if 'hTME19-2' in brain:
    if '01A' in slice_:
        A0[:3,-1] = np.array([0,0,800])
    if '01B' in slice_:
        A0[:3,-1] = np.array([0,-400,600])
    if '02' in slice_:
        A0[:3,-1] = np.array([0,-200,400])
    if '03' in slice_:
        A0[:3,-1] = np.array([0,0,0])
    if '04' in slice_:
        A0[:3,-1] = np.array([0,0,200])
    if '05' in slice_:
        A0[:3,-1] = np.array([0,0,-300])
    if '06' in slice_:
        A0[:3,-1] = np.array([0,300,-400])
    if '07' in slice_:
        A0[:3,-1] = np.array([0,100,-400])
    if '08' in slice_:
        A0[:3,-1] = np.array([0,-200,-600])

## Current shift

In [ ]:
# Shift in y: +y => Shift 10x down, -y => Shift 10x up
# Shift in z: +z => Shift 10x 
if '12m_MQC06-2' in brain:
    if '01' in slice_:
        A0[:3,-1] = np.array([0,-1200,-1000])
    if '02' in slice_: # Unable to get 10x to display (Likely due to abnormally large shift)
        A0[:3,-1] = np.array([0,-14000,38000])
        
# check it
XJ = np.meshgrid(*xJ,indexing='ij')
tform = emlddmm.Transform(A0,direction='b')
A0I = emlddmm.apply_transform_float(xI,I,tform.apply(XJ))

fig,ax = emlddmm.draw(np.concatenate((A0I,J*0)),xJ,vmin=0)
# fig,ax = emlddmm.draw(np.max(np.concatenate((A0I,J*0)),axis=-3,keepdims=True),xJ,vmin=0) # The MIP
fig.canvas.draw()

fig,ax = emlddmm.draw(np.concatenate((A0I*0,J)),xJ,vmin=0)
fig.canvas.draw()

fig,ax = emlddmm.draw(np.concatenate((A0I,J)),xJ,vmin=0)
fig.canvas.draw()

In [ ]:
raise Exception('Make initial guess in above cell')

# Registration

In [ ]:
# now we want to register
# we'll start with an affine registration
# device = 'cpu'
device = 'cuda:1'
config0 = {
    'device':device,
    'n_iter':500, 'downI':[2,2,2], 'downJ':[2,2,2],
     'priors':[0.9,0.05,0.05],'update_priors':False,
     'update_muA':0,'muA':[np.quantile(J,0.99)],
     'update_muB':0,'muB':[0.0],     
     'sigmaM':1.0,'sigmaB':2.0,'sigmaA':5.0,
     'order':1,'n_draw':50,'n_estep':3,'slice_matching':0,'v_start':1000,
     'eA':5e6,'A':A0,'full_outputs':True,'local_contrast':[8,16,16]
   }

I_ = np.concatenate((I,I**2))
out = emlddmm.emlddmm(xI=xI,I=I_,xJ=xJ,J=J, W0=W, **config0)
print("===========================================")
print("Finished registration with linear transform")
print("===========================================")
# Run with only a linear transform (no deformation)

In [ ]:
# second run, with deformation
imp.reload(emlddmm)
config1 = dict(config0)

config1['A'] = out['A']
config1['eA'] = config0['eA']*0.1
config1['a'] = 1000.0
config1['sigmaR'] = 5e4 # 1e4 gave really good results, but try 2e4, also good, I showed this in my slides
config1['n_iter']= 2000
config1['v_start'] = 0
config1['ev'] = 1e0
config1['v_res_factor'] = config1['a']/dI[0]/2/2 # what is the resolution of v, as a multiple of that in I

#config1['device'] = 'cpu'
# changing my inverses to double did not fix the issue
#config1['device'] = 'cuda:1'
#config1['local_contrast'] = [8,16,16]


out1 = emlddmm.emlddmm(xI=xI,I=I_,xJ=xJ,J=J, W0=W, **config1)
print("======================================")
print("Finished registration with deformation")
print("======================================")

In [ ]:
imp.reload(emlddmm)
# on the next run we do less downsampling
config2 = dict(config1)
config2['A'] = out1['A']
config2['n_iter'] = 1000
config2['v'] = out1['v']
config2['downI'] = [1,1,1]
config2['downJ'] = [1,1,1]
config2['ev'] = config1['ev']*0.5
config2['eA'] = config1['eA']*0.5
# there seems to be an issue with the initial velocity
# when I run this twice, I'm reusing it
out2 = emlddmm.emlddmm(xI=xI,I=I_,xJ=xJ,J=J, W0=W, **config2)
# the matching energy here is way way lower, why would that be?
print("============================================================")
print("Finished registration with deformation and less downsampling")
print("============================================================")

In [ ]:
# save the outputs
savename = f'transformation_outputs_{target_key}_dragonfly10to30_v00.npz'
np.savez(output_prefix+savename,out=out2)

out2['figI'].savefig(output_prefix + target_key + '_transformed.jpg',**figopts)
out2['figfI'].savefig(output_prefix + target_key + '_contrast.jpg',**figopts)
out2['figErr'].savefig(output_prefix + target_key + '_err.jpg',**figopts)

In [ ]:
fig = out2['figErr']
axs = fig.get_axes()
for ax in axs:
    ims = ax.get_images()
    for im in ims:
        im.set_cmap('twilight')
        clim = im.get_clim()
        lim = np.max(np.abs(clim))
        im.set_clim(np.array((-1,1))*lim)
fig.canvas.draw()
fig.savefig(output_prefix + target_key + '_err2.jpg',**figopts)
fig
fig.canvas.draw()

In [ ]:
# visualize overlap (Before optimization)

# check it
XJ = np.meshgrid(*xJ,indexing='ij')

tform = emlddmm.compose_sequence([
    emlddmm.Transform(A0,'b'),
                                  ],XJ)
AI = emlddmm.apply_transform_float(xI,I,tform)
fig,ax = emlddmm.draw(np.concatenate((AI,J)),xJ,vmin=0)
fig.canvas.draw()
fig.suptitle('error_before')
fig.savefig(output_prefix + target_key + '_err3_before.jpg',**figopts)

In [ ]:
# visualize overlap (After optimization)

# check it
XJ = np.meshgrid(*xJ,indexing='ij')

tform = emlddmm.compose_sequence([emlddmm.Transform(out2['A'],'b'),
                                  emlddmm.Transform(out2['v'],'b',domain=out2['xv'])],XJ)
AI = emlddmm.apply_transform_float(xI,I,tform)
fig,ax = emlddmm.draw(np.concatenate((AI,J)),xJ,vmin=0)
fig.canvas.draw()
fig.suptitle('error_after')
fig.savefig(output_prefix + target_key + '_err3_after.jpg',**figopts)